In [1]:
import openpyxl

In [2]:
%run MyFunctions.ipynb

In [3]:
## Import data
filename = "ResearchInChildAndAd_DATA_2018-12-14_1531.xlsx"
workbook = openpyxl.load_workbook(filename)
sheet1 = workbook["ResearchInChildAndAd_DATA_2018-"]
sheet2 = workbook["Tasks"]

In [4]:
## Make columns for new data
IGNORE = ['query', 'vs', 'r/o', 'rule out', 'versus']
KEYWORDS_DICT = {'ADHD': ['attention', 'adhd', 'add'], 
                 'ASD': ['autis', 'asd'],
                 'Anxiety': ['anxiety', 'panic', 'phobi'], 
                 'Bipolar': ['bipolar', 'mani'],
                 'Depression': ['depress', 'MDD', 'PDD', 'mood', 'dysthymi'], 
                 'EOS': ['schizo'], 
                 'FASD': ['fasd', 'fetal', 'alcohol'], 
                 'OCD': ['obsess', 'compuls', 'ocd'], 
                 'ODD': ['opposit', 'defiant', 'odd'], 
                 'Parent-Child': ['parent'], 
                 'Personality': ['cluster', 'personality', 'histrio', 'borderline'], 
                 'Psychosis': ['psychosis', 'psychotic'],
                 'SUD': ['substance', 'SUD', 'use disorder'], 
                 'Trauma': ['trauma', 'adjust', 'ptsd'], 
                 'eating': ['eating', 'anorexi', 'bulimi', 'binge'], 
                 'neurodevelopmental': ['learning', 'delay', 'neurodevelopment', 'intellectual', 'conduct', 'cerebral', 'palsy', 'chromosom'],
                 'somatic': ['somati'], 
                 'suicide': ['suicid']}
# dont forget handle fads and NOT personality for EOS
# read file in instead
ILLNESSES = []
for i in KEYWORDS_DICT.keys():
    ILLNESSES.append(i)

## Add columns
set_headers(sheet1, 'admission')
set_headers(sheet1, 'discharge')

In [5]:
## Create a dictionary of column names
ColNames = {}
Current  = 0
for COL in sheet1.iter_cols(1, sheet1.max_column):
    ColNames[COL[0].value] = Current
    Current += 1

In [6]:
## Module 1: Set binary values to 0
set_binaries_to_zero(sheet1, "admission", ColNames)
set_binaries_to_zero(sheet1, "discharge", ColNames)

In [7]:
## Module 2: Fill in main diagnosis
set_main_diagnosis(sheet1, "admission", ColNames)
set_main_diagnosis(sheet1, "discharge", ColNames)

In [8]:
## Module 3: Read Diagnosis, break into substrings, fill in
one_hot_encode_diagnoses(sheet1, 'admission', ColNames)
one_hot_encode_diagnoses(sheet1, 'discharge', ColNames)

In [9]:
## Output file
workbook.save("output-{}.xlsx".format(getTime()))